## Import packages

In [1]:
import pandas as pd
import numpy as np
import datetime

## Load data

In [2]:
activity_data = pd.read_excel('../Raw Data/ACTIVITY.xlsx')

In [3]:
activity_data.shape

(184, 5)

In [4]:
activity_data.head()

,date,steps,distance,calories,timeActive
0,2019-05-01,5433,3.71,145,01:22:00
1,2019-05-02,7765,5.92,254,01:29:00
2,2019-05-03,5485,3.94,154,01:15:00
3,2019-05-04,6521,4.44,177,01:40:00
4,2019-05-05,10372,7.14,262,02:19:00


In [5]:
sleep_data = pd.read_excel('../Raw Data/SLEEP.xlsx')

In [6]:
sleep_data.shape

(185, 8)

In [7]:
sleep_data.head()

,date,inBedFor,deepSleep,lightSleep,timeAwake,fellAsleepAt,wokeUpAt,sleepScore
0,2019-05-01,07:33:00,01:33:00,06:00:00,00:00:00,23:31:00,07:04:00,92
1,2019-05-02,08:18:00,02:43:00,05:35:00,00:02:00,22:17:00,06:37:00,93
2,2019-05-03,08:54:00,03:41:00,05:13:00,00:00:00,22:03:00,06:57:00,94
3,2019-05-04,07:43:00,03:43:00,04:00:00,00:18:00,00:06:00,08:07:00,88
4,2019-05-05,08:36:00,03:03:00,05:33:00,00:00:00,22:26:00,07:02:00,95


In [8]:
huawei_data = pd.read_excel('../Raw Data/HUAWEI.xlsx')

In [9]:
huawei_data.shape

(184, 4)

In [10]:
huawei_data.head()

,date,steps,distance,calories
0,2019-05-01,2104,1.59,72
1,2019-05-02,5601,4.23,191
2,2019-05-03,2921,2.21,99
3,2019-05-04,2956,2.23,101
4,2019-05-05,6337,4.79,216


## Data preparation

### Merging data

In [ ]:
activity_data['date'] = pd.to_datetime(activity_data['date'])
sleep_data['date'] = pd.to_datetime(sleep_data['date'])

In [ ]:
merged_data_temp = pd.merge(activity_data, sleep_data, on='date', how='inner')
merged_data = pd.merge(merged_data_temp, huawei_data, on='date', how='inner')

In [ ]:
merged_data.shape

### Selecting and renaming columns

In [ ]:
merged_data = merged_data[['date','start','stop','deepSleepTime','shallowSleepTime','wakeTime',
                           'steps_x','distance_x','calories_x','steps_y','distance_y','calories_y']]

In [ ]:
merged_data = merged_data.rename(columns={'start':'startSleep','stop':'stopSleep',
                                          'steps_x':'stepsMiFit','distance_x':'distanceMiFit','calories_x':'caloriesMiFit',
                                          'steps_y':'stepsHuawei','distance_y':'distanceHuawei','calories_y':'caloriesHuawei',})

### Filtering and indexing data

In [ ]:
merged_data = merged_data[(merged_data['date'] > '2019-04-14') & (merged_data['date'] < '2019-06-16')]

In [ ]:
merged_data.set_index('date', inplace = True)

In [ ]:
merged_data.shape

### Fixing columns types

In [ ]:
merged_data['startSleep'] = pd.to_datetime(merged_data['startSleep'], unit='s')
merged_data['stopSleep'] = pd.to_datetime(merged_data['stopSleep'], unit='s')

In [ ]:
merged_data['startSleep'] = merged_data['startSleep'] + datetime.timedelta(hours=3)
merged_data['stopSleep'] = merged_data['stopSleep'] + datetime.timedelta(hours=3)

In [ ]:
merged_data['distanceHuawei'] = merged_data['distanceHuawei'] * 1000
merged_data['distanceHuawei'] = merged_data['distanceHuawei'].astype(int)

### Creating new features

In [ ]:
merged_data['weekday'] = merged_data.index.dayofweek

In [ ]:
merged_data['totalSleepTime'] = (merged_data['stopSleep'] - merged_data['startSleep']).dt.total_seconds() / 60
merged_data['totalSleepTime'] = merged_data['totalSleepTime'].astype(int)

In [ ]:
merged_data.shape

In [ ]:
merged_data.head()

## Data analysis

- Understand what % of time Huawei phone is idle compared to MI Fit Band

In [ ]:
merged_data['stepsHuawei'].sum() / merged_data['stepsMiFit'].sum()

In [ ]:
merged_data['distanceHuawei'].sum() / merged_data['distanceMiFit'].sum()

In [ ]:
merged_data['caloriesHuawei'].sum() / merged_data['caloriesMiFit'].sum()

- Understand if ratio between steps/distance/calories is the same for phone and band

In [ ]:
merged_data['distanceHuawei'].sum() / merged_data['stepsHuawei'].sum()

In [ ]:
merged_data['distanceMiFit'].sum() / merged_data['stepsMiFit'].sum()

In [ ]:
merged_data['distanceHuawei'].sum() / merged_data['caloriesHuawei'].sum()

In [ ]:
merged_data['distanceMiFit'].sum() / merged_data['caloriesMiFit'].sum()